In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [2]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0   449k      0 --:--:-- --:--:-- --:--:--  453k


In [3]:
import random
import numpy as np

seed = 1239129

# Set the random seed for reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
from sklearn.model_selection import train_test_split

test_data = pd.read_csv('datasets/pima_indians_imputed.csv', index_col=0).astype(float)

y = test_data.iloc[:, -1]

x_train, x_test = train_test_split(test_data, test_size=0.5, random_state=seed, stratify=y)
y_train = x_train.iloc[:, -1]  # Extract labels from the training split
x_train, x_val = train_test_split(x_train, test_size=0.1, random_state=seed, stratify=y_train)

x_train.to_csv('datasets/train.csv')
x_val.to_csv('datasets/val.csv')
x_test.to_csv('datasets/test.csv')

In [5]:
import torch
from sklearn.metrics import matthews_corrcoef

def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_metrics(model, data_loader):
    correct = 0
    total = 0
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    
    all_true_labels = []
    all_predicted_labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Collect all predictions and true labels for MCC
            all_true_labels.extend(true_labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)
            
            # Calculate TP, FP, FN, TN
            true_positives += ((predicted_labels == 1) & (true_labels == 1)).sum().item()
            false_positives += ((predicted_labels == 1) & (true_labels == 0)).sum().item()
            false_negatives += ((predicted_labels == 0) & (true_labels == 1)).sum().item()
            true_negatives += ((predicted_labels == 0) & (true_labels == 0)).sum().item()
    
    accuracy = correct / total if total > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    balanced_accuracy = 0.5 * (recall + (true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0))
    mcc = matthews_corrcoef(all_true_labels, all_predicted_labels)
    tnr = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0  # True Negative Rate
    fpr = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0  # False Positive Rate
    fnr = false_negatives / (false_negatives + true_positives) if (false_negatives + true_positives) > 0 else 0  # False Negative Rate
    tpr = recall  # True Positive Rate is the same as recall

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}, True Negatives: {true_negatives}, Total: {total}")
    print()
    print(f"A (Accuracy): {accuracy:.4f}")
    print(f"P (Precision): {precision:.4f}")
    print(f"R (Recall): {recall:.4f}")
    print(f"F1 (F1 Score): {f1_score:.4f}")
    print(f"BA (Balanced Accuracy): {balanced_accuracy:.4f}")
    print(f"MCC (Matthews Correlation Coefficient): {mcc:.4f}")
    print(f"TNR (True Negative Rate): {tnr:.4f}")
    print(f"FPR (False Positive Rate): {fpr:.4f}")
    print(f"FNR (False Negative Rate): {fnr:.4f}")
    print(f"TPR (True Positive Rate): {tpr:.4f}")
    model.train()

In [6]:
kb = KnowledgeBase("with_logic.yaml")

Using device: cpu


In [7]:
kb.predicates["Diabetic"]

<LogicTensorNetwork: ['person']>

In [8]:
fine_tune = True

In [9]:
if fine_tune:
    model = kb.predicates["Diabetic"]

    criteria = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

    patience = 5
    min_delta = 0.0
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(50):
        model.train()
        total_loss = 0.0
        num_batches = 0
        
        # Training loop
        for data, labels in kb.loaders[0]:
            optimizer.zero_grad()
            predictions = model(data)
            loss = criteria(predictions, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1

        avg_train_loss = total_loss / num_batches

        model.eval()
        total_val_loss = 0.0
        num_val_batches = 0

        for data, labels in kb.val_loaders[0]:
            with torch.no_grad():
                predictions = model(data)
                val_loss = criteria(predictions, labels)
                total_val_loss += val_loss.item()
                num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        # Early stopping logic
        if avg_val_loss + min_delta < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early Stopping at: Epoch {epoch + 1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
            break

        if epoch % 10 == 0:
            print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

        model.train()

Epoch 1, Train Loss: 0.667559, Val Loss: 0.666479
Epoch 11, Train Loss: 0.422746, Val Loss: 0.511885
Epoch 21, Train Loss: 0.343546, Val Loss: 0.497202
Early Stopping at: Epoch 28, Train Loss: 0.324318, Val Loss: 0.502187


In [10]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 97, False Positives: 9, False Negatives: 23, True Negatives: 216, Total: 345

A (Accuracy): 0.9072
P (Precision): 0.9151
R (Recall): 0.8083
F1 (F1 Score): 0.8584
BA (Balanced Accuracy): 0.8842
MCC (Matthews Correlation Coefficient): 0.7932
TNR (True Negative Rate): 0.9600
FPR (False Positive Rate): 0.0400
FNR (False Negative Rate): 0.1917
TPR (True Positive Rate): 0.8083


In [11]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 81, False Positives: 35, False Negatives: 53, True Negatives: 215, Total: 384

A (Accuracy): 0.7708
P (Precision): 0.6983
R (Recall): 0.6045
F1 (F1 Score): 0.6480
BA (Balanced Accuracy): 0.7322
MCC (Matthews Correlation Coefficient): 0.4822
TNR (True Negative Rate): 0.8600
FPR (False Positive Rate): 0.1400
FNR (False Negative Rate): 0.3955
TPR (True Positive Rate): 0.6045


In [12]:
if fine_tune:
    kb.optimize(num_epochs=1, log_steps=10, lr=0.000001, early_stopping=True, patience=5)
else:
    kb.optimize(num_epochs=150, log_steps=10, lr=0.00001, early_stopping=False, patience=5)

Tensor1:  tensor([23.1624, 41.5962, 30.5168, 36.9161, 22.2073, 39.1129, 32.0450, 44.0795,
        28.2245, 30.8988, 32.4270, 32.0450, 44.7481, 29.7527, 21.6342, 32.9046,
        32.8091, 34.5283, 39.1129, 37.2026, 31.2809, 27.7470, 35.8655, 39.4949,
        34.3373, 36.1520, 49.2371, 33.8597, 40.9276, 20.1060, 31.6629, 24.7861])
Tensor2:  tensor(26., requires_grad=True)
Tensor1 Normalized:  tensor([0.1049, 0.7377, 0.3574, 0.5770, 0.0721, 0.6525, 0.4098, 0.8230, 0.2787,
        0.3705, 0.4230, 0.4098, 0.8459, 0.3311, 0.0525, 0.4393, 0.4361, 0.4951,
        0.6525, 0.5869, 0.3836, 0.2623, 0.5410, 0.6656, 0.4885, 0.5508, 1.0000,
        0.4721, 0.7148, 0.0000, 0.3967, 0.1607], grad_fn=<DivBackward0>)
Tensor2 Normalized:  tensor([0.2023], grad_fn=<DivBackward0>)
Result:  tensor([0.6642, 0.0230, 0.2525, 0.0677, 0.7133, 0.0411, 0.1896, 0.0128, 0.3695,
        0.2356, 0.1759, 0.1896, 0.0109, 0.2887, 0.7406, 0.1599, 0.1630, 0.1141,
        0.0411, 0.0635, 0.2194, 0.3966, 0.0854, 0.0376, 0.1188

In [13]:
[op.k for op in kb.converter.visitor.comparision_operators]

[Parameter containing:
 tensor([7.0000], requires_grad=True),
 Parameter containing:
 tensor([7.0000], requires_grad=True),
 Parameter containing:
 tensor([7.0000], requires_grad=True),
 Parameter containing:
 tensor([7.0000], requires_grad=True)]

In [14]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 95, False Positives: 12, False Negatives: 25, True Negatives: 213, Total: 345

A (Accuracy): 0.8928
P (Precision): 0.8879
R (Recall): 0.7917
F1 (F1 Score): 0.8370
BA (Balanced Accuracy): 0.8692
MCC (Matthews Correlation Coefficient): 0.7602
TNR (True Negative Rate): 0.9467
FPR (False Positive Rate): 0.0533
FNR (False Negative Rate): 0.2083
TPR (True Positive Rate): 0.7917


In [15]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 80, False Positives: 35, False Negatives: 54, True Negatives: 215, Total: 384

A (Accuracy): 0.7682
P (Precision): 0.6957
R (Recall): 0.5970
F1 (F1 Score): 0.6426
BA (Balanced Accuracy): 0.7285
MCC (Matthews Correlation Coefficient): 0.4756
TNR (True Negative Rate): 0.8600
FPR (False Positive Rate): 0.1400
FNR (False Negative Rate): 0.4030
TPR (True Positive Rate): 0.5970
